<a href="https://colab.research.google.com/github/nagusubra/Solar_panel_dust_detection/blob/main/Solar_panel_dust_detection_compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install libraries and modules

In [1]:
!pip install -q tensorflow-model-optimization
!pip install openpyxl
!pip install xlsxwriter

import xlsxwriter
import openpyxl
from openpyxl import Workbook
from openpyxl.drawing.image import Image

import tempfile
from tensorflow import keras
import tensorflow_model_optimization as tfmot

from tqdm import tqdm_notebook as tqdm
import os
import time
import pickle
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 54.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.2 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 KB 5.7 MB/s eta 0:00:00


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import binary_crossentropy
import tensorflow_datasets as tfds
import zipfile

In [8]:
# mounting google drive (if you are using Colab)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Evaluation function

In [9]:
# Evaluate Model Size
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)
  return os.path.getsize(zipped_file)


def evaluate_model(model_path, model_info, val_dataset):

  # Evaluate test accuracy and test loss
  model = tf.keras.models.load_model(model_path)
  test_loss, test_acc = model.evaluate(val_dataset, verbose=0)

  # Evaluate Model Size
  model_size = get_gzipped_model_size(model_path)

  # Evaluate Inference Time
  startTime = time.time()
  prediction = model.predict(val_dataset)
  executionTime = (time.time() - startTime)/len(val_dataset)

  # Print
  print('\nModel Accuracy:', test_acc*100, '%')
  print("Model Size: %.2f bytes" % (model_size))
  print("Inference Time is: ", executionTime, "s")

  # Build Evalution dataframe
  evulation_dict = {
                      "Evaluation type": "Evualation",
                      "Model Information": model_info,
                      "Accuracy": str(test_acc*100) + " %",
                      "Loss": str(test_loss*100) + " %",
                      "Model Size": str(model_size) + " bytes",
                      "Inference Time": str(executionTime) + " sec"
                    }
  
  evulation_df = pd.DataFrame.from_dict(evulation_dict, orient='index').reset_index()


  return test_acc, model_size, executionTime, evulation_df


def evaluate_model_without_saving_stats(model, model_path, model_info, val_dataset):

  # Evaluate test accuracy and test loss
  # model = tf.keras.models.load_model(model_path)
  test_loss, test_acc = model.evaluate(val_dataset, verbose=0)

  # Evaluate Model Size
  model_size = get_gzipped_model_size(model_path)

  # Evaluate Inference Time
  startTime = time.time()
  prediction = model.predict(val_dataset)
  executionTime = (time.time() - startTime)/len(val_dataset)

  # Print
  print('\nModel Accuracy:', test_acc*100, '%')
  print("Model Size: %.2f bytes" % (model_size))
  print("Inference Time is: ", executionTime, "s")

  # Build Evalution dataframe
  evulation_dict = {
                      "Evaluation type": "Evualation",
                      "Model Information": model_info,
                      "Accuracy": str(test_acc*100) + " %",
                      "Loss": str(test_loss*100) + " %",
                      "Model Size": str(model_size) + " bytes",
                      "Inference Time": str(executionTime) + " sec"
                    }
  
  evulation_df = pd.DataFrame.from_dict(evulation_dict, orient='index').reset_index()


  return test_acc, model_size, executionTime, evulation_df




# research paper evaluation function
def evaluate_research_paper_with_model_path(model_path):
  # model_path = "/content/models/solnet.hdf5"
  solnet = load_model(model_path, compile=False)
  history = solnet.history()
  plt.plot(history.history['loss'])
  plt.plot(history.history['acc'])
  plt.title('acc loss vs epoch')
  plt.xlabel('epoch')
  plt.legend(['loss', 'acc'], loc='upper left')
  plt.show()

def evaluate_research_paper_with_model(model):
  history = model.history()
  plt.plot(history.history['loss'])
  plt.plot(history.history['acc'])
  plt.title('acc loss vs epoch')
  plt.xlabel('epoch')
  plt.legend(['loss', 'acc'], loc='upper left')
  plt.show()

# Load base model and Evaluate


In [10]:
# final_model_path = '/content/drive/MyDrive/Solar_panel_dust_detection/final_base_model/solnetOriginal_final_base_model.hdf5'
final_model_path = '/content/drive/MyDrive/Solar_panel_dust_detection/final_base_model/final_base_model.h5'
final_model = tf.keras.models.load_model(final_model_path)

In [11]:
batch_size = 32
#location = 'dataset/'
location = "/content/drive/MyDrive/Solar_panel_dust_detection/dataset_1"
label_mode = 'binary'
seed = 10 #changed for each fold made manually

class_names = ['clean', 'dirty']
in_size = [227, 227, 3]

tr_dataset = image_dataset_from_directory(directory=location, label_mode= label_mode, class_names=class_names,
                                          seed=seed, labels='inferred', image_size=in_size[:-1], 
                                          subset = 'training', batch_size=batch_size, validation_split=.2)

val_dataset = image_dataset_from_directory(directory=location, label_mode= label_mode, class_names=class_names,
                                          seed=seed, labels='inferred', image_size=in_size[:-1],
                                          subset = 'validation', batch_size=batch_size, validation_split=.2)


Found 1440 files belonging to 2 classes.
Using 1152 files for training.
Found 1440 files belonging to 2 classes.
Using 288 files for validation.


In [12]:
test_acc, model_size, executionTime, evulation_df = evaluate_model_without_saving_stats(final_model, final_model_path, "#0", val_dataset)
print(test_acc)

9/9 [==============================] - 25s 38ms/step

Model Accuracy: 96.18055820465088 %
Model Size: 648760230.00 bytes
Inference Time is:  2.788432492150201 s
0.9618055820465088


In [13]:
evulation_df

,index,0
0,Evaluation type,Evualation
1,Model Information,#0
2,Accuracy,96.18055820465088 %
3,Loss,11.30877360701561 %
4,Model Size,648760230 bytes
5,Inference Time,2.788432492150201 sec


# Compression

## One-shot pruning

## Iterative pruning

In [14]:
def iterative_pruning(model, initial_sparsity, final_sparsity, begin_step, end_step, train_images, train_labels, epochs):
  prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

  # Define model for pruning.
  pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=initial_sparsity,
        final_sparsity=final_sparsity, begin_step=begin_step, end_step=end_step, frequency=100)
  }

  pruned_model = prune_low_magnitude(model, **pruning_params)

  # `prune_low_magnitude` requires a recompile.
  optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
  pruned_model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])


  callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
  ]

  pruned_model.fit(train_images, train_labels, epochs=epochs, validation_split=0.1,
                    callbacks=callbacks)

  # Strip pruning wrappers
  stripped_pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

  return pruned_model, stripped_pruned_model


def iterative_pruning2(model, initial_sparsity, final_sparsity, begin_step, end_step, tdata_loader,vdata_loader, epochs):
  prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

  # Define model for pruning.
  pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=initial_sparsity,
          final_sparsity=final_sparsity, begin_step=begin_step, end_step=end_step, frequency=100)
  }

  pruned_model = prune_low_magnitude(model, **pruning_params)

  # `prune_low_magnitude` requires a recompile.
  optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
  pruned_model.compile(optimizer=Adam(.0001, .8, .9),
      loss=binary_crossentropy,
      metrics=['accuracy'])

  callbacks = [
      tfmot.sparsity.keras.UpdatePruningStep(),
  ]

  pruned_model.fit(tdata_loader, epochs=epochs, validation_data=vdata_loader, callbacks=callbacks)

  # Strip pruning wrappers
  stripped_pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

  return pruned_model, stripped_pruned_model

In [15]:
batch_size = 10000
#location = 'dataset/'
location = "/content/drive/MyDrive/Solar_panel_dust_detection/dataset_1"
label_mode = 'binary'
seed = 10 #changed for each fold made manually

class_names = ['clean', 'dirty']
in_size = [227, 227, 3]

tr_dataset_for_loading = image_dataset_from_directory(directory=location, label_mode= label_mode, class_names=class_names,
                                          seed=seed, labels='inferred', image_size=in_size[:-1], 
                                          subset = 'training', batch_size=batch_size, validation_split=.2)

val_dataset_for_loading = image_dataset_from_directory(directory=location, label_mode= label_mode, class_names=class_names,
                                          seed=seed, labels='inferred', image_size=in_size[:-1],
                                          subset = 'validation', batch_size=batch_size, validation_split=.2)


Found 1440 files belonging to 2 classes.
Using 1152 files for training.
Found 1440 files belonging to 2 classes.
Using 288 files for validation.


In [16]:
type(tr_dataset_for_loading)

tensorflow.python.data.ops.batch_op._BatchDataset

In [63]:
# frquency = 100
# target sparsity = 50%
# sparisty for each pruning tep = 50%/100 = 0.05%
# Total Number of Training Samples = 1440
# batch size = 32
# epochs = 10

# possible start and end for the iterative pruning schedule
# Number of learning steps = (1440/32)*3 = 135 (15, 120)
# Number of learning steps = (1440/32)*4 = 180 (, )
# Number of learning steps = (1440/32)*5 = 225 (, )
# Number of learning steps = (1440/32)*6 = 270 (, )

# Number of learning steps = (1440/32)*10 = 450 (45, 400)
# Number of learning steps = (1440/32)*20 = 900 (90, 800)



# Number of learning steps = (1440/32)*30 = 1350 (135, 1200)

# iterative_pruning(
                  #     model, 
                  #     initial_sparsity = 0, 
                  #     final_sparsity   = 0.9, 
                  #     begin_step       = 0, 600  (~10% of 5625) # pruning early on the pruning schedule to get optimal model size
                  #     end_step         = ?, 4500 (~80% of 5625) # pruning only till 80% so that model can still have enough leanring steps to learn and build networks for the data
                  #     train_images, 
                  #     train_labels, 
                  #     epochs           = 3, no change ? since the Number of Steps per Epoch = (Total Number of Training Samples = ?) / (Batch Size = ?) 
                  # )



# pruned_model_saprsity_50, stripped_pruned_model_saprsity_50 = iterative_pruning2(final_model, 0, 0.5, 45, 400, tr_dataset, val_dataset, 10)
pruned_model_saprsity_50, stripped_pruned_model_saprsity_50 = iterative_pruning2(final_model, 0, 0.5, 90, 800, tr_dataset, val_dataset, 20)


# pruned_model_saprsity_50, stripped_pruned_model_saprsity_50 = iterative_pruning(final_model, 0, 0.5, 15, 120, x_train_images, x_train_labels, 3)
# pruned_model_saprsity_50, stripped_pruned_model_saprsity_50 = iterative_pruning(final_model, 0, 0.9, 45, 400, x_train_images, x_train_labels, 10)

# pruned_model_saprsity_50, stripped_pruned_model_saprsity_50 = iterative_pruning(final_model, 0, 0.5, 135, 1200, x_train_images, x_train_labels, 30)


Epoch 1/20
36/36 [==============================] - 132s 3s/step - loss: 0.0800 - accuracy: 0.9861 - val_loss: 7.6558 - val_accuracy: 0.5833
Epoch 2/20
36/36 [==============================] - 123s 3s/step - loss: 0.0559 - accuracy: 0.9852 - val_loss: 1.0496 - val_accuracy: 0.8403
Epoch 3/20
36/36 [==============================] - 123s 3s/step - loss: 0.1132 - accuracy: 0.9809 - val_loss: 15.4979 - val_accuracy: 0.5972
Epoch 4/20
36/36 [==============================] - 124s 3s/step - loss: 0.0905 - accuracy: 0.9878 - val_loss: 1.4052 - val_accuracy: 0.8403
Epoch 5/20
36/36 [==============================] - 119s 3s/step - loss: 0.1005 - accuracy: 0.9844 - val_loss: 0.8120 - val_accuracy: 0.9132
Epoch 6/20
36/36 [==============================] - 121s 3s/step - loss: 0.1038 - accuracy: 0.9826 - val_loss: 2.6321 - val_accuracy: 0.7431
Epoch 7/20
36/36 [==============================] - 119s 3s/step - loss: 0.0569 - accuracy: 0.9896 - val_loss: 0.9422 - val_accuracy: 0.9201
Epoch 8/20
3

In [64]:
stripped_pruned_model_saprsity_50.save('stripped_pruned_model_saprsity_50.h5')

In [65]:
def print_model_weights_sparsity(model):

    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Wrapper):
            weights = layer.trainable_weights
        else:
            weights = layer.weights
        for weight in weights:
            if "kernel" not in weight.name or "centroid" in weight.name:
                continue
            weight_size = weight.numpy().size
            zero_num = np.count_nonzero(weight == 0)
            print(
                f"{weight.name}: {zero_num/weight_size:.2%} sparsity ",
                f"({zero_num}/{weight_size})",
            )


print_model_weights_sparsity(pruned_model_saprsity_50)

conv2d_10/kernel:0: 49.81% sparsity  (17359/34848)
conv2d_11/kernel:0: 49.81% sparsity  (306058/614400)
conv2d_12/kernel:0: 49.81% sparsity  (440723/884736)
conv2d_13/kernel:0: 49.81% sparsity  (661084/1327104)
conv2d_14/kernel:0: 49.81% sparsity  (440723/884736)
dense_6/kernel:0: 49.81% sparsity  (18804176/37748736)
dense_7/kernel:0: 49.81% sparsity  (8357412/16777216)
dense_8/kernel:0: 49.80% sparsity  (2040/4096)


In [66]:
test_loss_pruned_50, test_acc_pruned_50 = pruned_model_saprsity_50.evaluate(val_dataset, verbose=0)

In [67]:
startTime = time.time()
prediction = pruned_model_saprsity_50.predict(val_dataset)
executionTimePruned50 = (time.time() - startTime)/len(val_dataset)

9/9 [==============================] - 22s 40ms/step


In [68]:
pruned_model_size = get_gzipped_model_size('stripped_pruned_model_saprsity_50.h5')

In [69]:
print('\nPruned Model Accuracy:', test_acc_pruned_50*100, '%')
print("Pruned Model Size: %.2f bytes" % (pruned_model_size))
print("Pruned Inference Time is", executionTimePruned50, "s")


Pruned Model Accuracy: 95.13888955116272 %
Pruned Model Size: 129522911.00 bytes
Pruned Inference Time is 2.413063473171658 s


In [70]:
pruned_test_acc, pruned_model_size, pruned_executionTime, pruned_evulation_df = evaluate_model_without_saving_stats(pruned_model_saprsity_50, 'stripped_pruned_model_saprsity_50.h5', "#1", val_dataset)
print(pruned_test_acc)

9/9 [==============================] - 21s 37ms/step

Model Accuracy: 95.13888955116272 %
Model Size: 129522911.00 bytes
Inference Time is:  2.3474700185987682 s
0.9513888955116272


In [71]:
pruned_evulation_df

,index,0
0,Evaluation type,Evualation
1,Model Information,#1
2,Accuracy,95.13888955116272 %
3,Loss,83.02435278892517 %
4,Model Size,129522911 bytes
5,Inference Time,2.3474700185987682 sec


In [72]:
evulation_df

,index,0
0,Evaluation type,Evualation
1,Model Information,#0
2,Accuracy,96.18055820465088 %
3,Loss,11.30877360701561 %
4,Model Size,648760230 bytes
5,Inference Time,2.788432492150201 sec


In [73]:
pruned_model_saprsity_50.save('/content/drive/MyDrive/Solar_panel_dust_detection/final_base_model/pruned_model_saprsity_50.h5')
stripped_pruned_model_saprsity_50.save('/content/drive/MyDrive/Solar_panel_dust_detection/final_base_model/stripped_pruned_model_saprsity_50.h5')

## Quantization

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_to_quantize)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
fp16_quantized_tflite_model = converter.convert()

with open('fp16_quantized_tflite_model.tflite', 'wb') as f:
  f.write(fp16_quantized_tflite_model)
interpreter_fp16_quant = tf.lite.Interpreter(model_path=str('fp16_quantized_tflite_model.tflite'))
interpreter_fp16_quant.allocate_tensors()

evaluate_model(interpreter_fp16_quant, 'fp16_quantized_tflite_model.tflite')